In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
from sklearn.model_selection import train_test_split
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [50]:
def is_van_gogh(artist_name: str) -> int:
    return int("van-gogh" in artist_name.lower())


In [ ]:
import wandb
# שים לב לשם הפרמטר key
wandb.login(key="16d1bc863b28f81253ac0ee253b453393791a7e1")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

 # ****Data**** ****Preparation**** 

In [52]:
import os
import pandas as pd

base_dir = "/kaggle/input/wikiart/Post_Impressionism"

records = []

for fname in os.listdir(base_dir):
    if not fname.lower().endswith((".jpg", ".png")):
        continue

    artist = fname.split("_")[0] #takes the first item of the split- the name of artist

    records.append({
        "filepath": os.path.join(base_dir, fname),
        "artist": artist,
        "is_van_gogh": int("van-gogh" in artist.lower())
    })

df = pd.DataFrame(records)


##  **check** **the** **dataframe**

In [53]:
df.head()

,filepath,artist,is_van_gogh
0,/kaggle/input/wikiart/Post_Impressionism/paul-...,paul-cezanne,0
1,/kaggle/input/wikiart/Post_Impressionism/suzan...,suzanne-valadon,0
2,/kaggle/input/wikiart/Post_Impressionism/alber...,albert-marquet,0
3,/kaggle/input/wikiart/Post_Impressionism/vince...,vincent-van-gogh,1
4,/kaggle/input/wikiart/Post_Impressionism/paul-...,paul-gauguin,0


In [54]:
print (df["is_van_gogh"].value_counts())
print (df[df["is_van_gogh"] == 1].head())

is_van_gogh
0    5445
1    1005
Name: count, dtype: int64
                                             filepath            artist  \
3   /kaggle/input/wikiart/Post_Impressionism/vince...  vincent-van-gogh   
9   /kaggle/input/wikiart/Post_Impressionism/vince...  vincent-van-gogh   
16  /kaggle/input/wikiart/Post_Impressionism/vince...  vincent-van-gogh   
17  /kaggle/input/wikiart/Post_Impressionism/vince...  vincent-van-gogh   
18  /kaggle/input/wikiart/Post_Impressionism/vince...  vincent-van-gogh   

    is_van_gogh  
3             1  
9             1  
16            1  
17            1  
18            1  


## *splitting the dataframe, and check it*

In [55]:
from sklearn.model_selection import train_test_split

# Step 1: Initial split - 70% for training and 30% for a temporary set (temp_df)
train_df, temp_df = train_test_split(
    df,
    test_size=0.3,           # Reserves 30% of the data for validation and testing
    stratify=df["is_van_gogh"], # Ensures class proportions are preserved across splits
    random_state=42          # Sets a seed for reproducibility
)

# Step 2: Second split - Divide the 30% temporary set equally into validation and test sets
# This results in 15% of the total data for validation and 15% for testing
val_df, test_df = train_test_split(
    temp_df,
    test_size=0.5,           # Splits the 30% into two equal 15% parts
    stratify=temp_df["is_van_gogh"], # Maintains class balance in the remaining subsets
    random_state=42
)

In [56]:
# Calculate the total number of rows
total_count = len(df)

# Print percentages of each split relative to the original dataset
print(f"Train set: {len(train_df)} rows ({len(train_df) / total_count:.1%})")
print(f"Val set:   {len(val_df)} rows ({len(val_df) / total_count:.1%})")
print(f"Test set:  {len(test_df)} rows ({len(test_df) / total_count:.1%})")

print("-" * 30)

# Print class distribution (counts) to verify stratification
print("Class distribution per split:")
print("\nTrain:\n", train_df["is_van_gogh"].value_counts(normalize=True)) # normalize=True gives ratios
print("\nVal:\n",   val_df["is_van_gogh"].value_counts(normalize=True))
print("\nTest:\n",  test_df["is_van_gogh"].value_counts(normalize=True))

Train set: 4515 rows (70.0%)
Val set:   967 rows (15.0%)
Test set:  968 rows (15.0%)
------------------------------
Class distribution per split:

Train:
 is_van_gogh
0    0.844075
1    0.155925
Name: proportion, dtype: float64

Val:
 is_van_gogh
0    0.844881
1    0.155119
Name: proportion, dtype: float64

Test:
 is_van_gogh
0    0.844008
1    0.155992
Name: proportion, dtype: float64


### Trasfomation class for VGG & AlexNet standard

In [57]:
from torchvision import transforms
"""
1 Standard transformations for models pre-trained on ImageNet ( VGG, AlexNet)
2 Resizes the image to 224x224, the standard input size for VGG and AlexNet
3 Converts the image to a PyTorch tensor and scales pixels to the [0, 1] range
4 Normalizes the tensor using ImageNet's mean and standard deviation.
5 Essential for maintaining consistency with VGG and AlexNet pre-trained weights.


augmentaion used: 
- RandomResizedCrop: Crops a random portion of the image and resizes it to 224x224. 
  This helps the model become invariant to scale and object position.
- RandomHorizontalFlip: Flips the image horizontally with a 50% probability. 
  Useful for art since composition can be mirrored without losing stylistic meaning.
- RandomRotation: Rotates the image by up to 10 degrees to handle slight tilts.
- ColorJitter: Randomly changes brightness, contrast, saturation, and hue. 
  This is crucial for Van Gogh's work as it makes the model robust to different lighting 
  conditions and minor color shifts in art photography.
- ToTensor & Normalize: Standard ImageNet preprocessing for model compatibility.
"""
# will be used to trasform the images to normalized tensor object  --> base_transform(un-normalized picture) and the Augmentaions

train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=10),
    transforms.ColorJitter(
        brightness=0.2,
        contrast=0.2,
        saturation=0.2,
        hue=0.05
    ),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [58]:
# Like the Train transforom but without the augmentaions

eval_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

### Making Dataset Object

In [59]:
from torch.utils.data import Dataset
from PIL import Image

class VanGoghDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df.reset_index(drop=True)   
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]

        image = Image.open(row["filepath"]).convert("RGB")
        label = row["is_van_gogh"]

        if self.transform:
            image = self.transform(image)

        return image, label


In [60]:
train_dataset = VanGoghDataset(train_df, transform=train_transform)
val_dataset   = VanGoghDataset(val_df,   transform=eval_transform)
test_dataset  = VanGoghDataset(test_df,  transform=eval_transform)


In [61]:
train_dataset[0][0]

tensor([[[-2.1179, -2.1179, -2.1179,  ...,  0.7933,  0.7933, -2.1179],
         [ 0.4166,  0.5022,  0.6734,  ...,  0.7933,  0.8104, -2.1179],
         [ 0.5707,  0.6734,  0.7591,  ...,  0.7933,  0.8104, -2.1179],
         ...,
         [-2.1179, -0.8849, -1.1075,  ..., -0.6281, -0.4739, -0.3883],
         [-2.1179, -0.7308, -1.0219,  ..., -0.6794, -0.6109, -0.6109],
         [-2.1179, -0.9705, -1.0219,  ..., -2.1179, -2.1179, -2.1179]],

        [[-2.0357, -2.0357, -2.0357,  ...,  0.8529,  0.8354, -2.0357],
         [ 0.2927,  0.2227,  0.4328,  ...,  0.8880,  0.8704, -2.0357],
         [ 0.3803,  0.4328,  0.6078,  ...,  0.9930,  0.9405, -2.0357],
         ...,
         [-2.0357, -0.8627, -1.0378,  ..., -0.4776, -0.2150, -0.1275],
         [-2.0357, -0.7402, -1.0028,  ..., -0.3901, -0.2850, -0.3375],
         [-2.0357, -1.0203, -1.0378,  ..., -2.0357, -2.0357, -2.0357]],

        [[-1.8044, -1.8044, -1.8044,  ...,  0.3916,  0.3742, -1.8044],
         [-0.2881, -0.3578, -0.0092,  ...,  0

In [62]:
import torch
# sanity check
# Retrieve images only (index 0 of the returned tuple)
img1 = test_dataset[0][0]
img2 = test_dataset.__getitem__(0)[0]
print(torch.equal(img1, img2)) 



True


### Making  DataLoader obj to later use in our model

In [63]:
from torch.utils.data import DataLoader

# DataLoader for the training set
train_loader = DataLoader(
    train_dataset,
    batch_size=32,      # Number of images per training step
    shuffle=True,       # Shuffle the data to prevent the model from learning the order
    num_workers=2       # Use 2 parallel CPU processes to load data faster
)

# DataLoader for the validation set
val_loader = DataLoader(
    val_dataset,
    batch_size=32,      # Typically matches the training batch size
    shuffle=False,      # No need to shuffle during evaluation
    num_workers=2
)

# Model Training & Tuning


In [64]:
import torch

# הגדרת המכשיר - אם יש GPU ב-Kaggle, הוא ישתמש בו
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Working on: {device}") # ב-Kaggle זה צריך להדפיס cuda


Working on: cuda


In [65]:
from torchvision import models
from torch import nn

model = models.vgg19(pretrained=True)

model.classifier[-1] = nn.Linear(
    model.classifier[-1].in_features, 2
)

model = model.to(device)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [66]:
# Print only the last layer of the classifier
print(model.classifier[-1])

Linear(in_features=4096, out_features=2, bias=True)


In [67]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [68]:
def train_one_epoch(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()    # each grad will start with grad = zero
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        
        # בדיקה אם יש ריצה פעילה ב-W&B כדי למנוע את השגיאה שקיבלת
        if wandb.run is not None:
            wandb.log({"batch_loss": loss.item()})

    return total_loss / len(loader)

@torch.no_grad()
def eval_one_epoch(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0

    for images, labels in loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        total_loss += loss.item()
        preds = outputs.argmax(dim=1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)
        
        if wandb.run is not None:
            wandb.log({"val_batch_loss": loss.item()})
            
    return total_loss / len(loader), correct / total

In [69]:
run = wandb.init(
    project="VanGogh_Classifier", 
    name="Initial_Test",
    config={
        "architecture": "VGG19",
        "epochs": 3
    }
)


for epoch in range(num_epochs):
    train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
    val_loss, val_acc = eval_one_epoch(model, val_loader, criterion)
    # ...

run.finish()

batch_loss,▆█▃▄▄▂▅▅▃▄▄▃▃▅▅▂▄▃▁▁▄▅▄▃▂▄▃▂▃▂▃▆▂▁▂▃▂▂▂▅
val_batch_loss,▅▆▅▇▅▇▆▃▆█▆▇█▄▅▆▆▂▅█▁▁▃▃▂▅▂▁▆▃▄▂▃▃▆▄▂▃▄▁
batch_loss,0.04852
val_batch_loss,0.08983


In [73]:
import optuna
import wandb
from torch.utils.data import DataLoader

def objective(trial):
    # 1. בחירת היפר-פרמטרים על ידי Optuna
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "SGD"])
    batch_size = trial.suggest_categorical("batch_size", [16, 32, 64])
    
    # הגדרת ה-DataLoaders בתוך הפונקציה כדי להשתמש ב-batch_size שנבחר
    curr_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2) # shuffe is important between each epoch
    curr_val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2)
    
    # 2. אתחול ריצה ב-Weights & Biases (חובה לפי ההנחיות)
    run = wandb.init(
        project="VanGogh_Classifier",
        name=f"trial_{trial.number}",
        config={
            "lr": lr, 
            "optimizer": optimizer_name, 
            "batch_size": batch_size,
            "model": "VGG19"
        },
        reinit=True
    )

    # 3. בניית המודל - בכל פעם מחדש עבור הניסיון הנוכחי
    model = models.vgg19(pretrained=True)
    model.classifier[-1] = nn.Linear(model.classifier[-1].in_features, 2)
    model = model.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = getattr(torch.optim, optimizer_name)(model.parameters(), lr=lr)
    
    # 4. לולאת אימון לפי מספר Epochs מוגדר
    num_epochs = 5 
    for epoch in range(num_epochs):
        # אימון והערכה בעזרת הפונקציות שכתבת
        train_loss = train_one_epoch(model, curr_train_loader, optimizer, criterion)
        val_loss, val_acc = eval_one_epoch(model, curr_val_loader, criterion)
        
        # דיווח מדדים ל-W&B בסוף כל Epoch
        wandb.log({
            "epoch": epoch, 
            "train_loss": train_loss, 
            "val_loss": val_loss,
            "val_acc": val_acc
        })
        
        # דיווח ל-Optuna ובדיקת "גיזום" (Pruning) לחיסכון בזמן
        trial.report(val_acc, epoch)
        if trial.should_prune():
            run.finish()
            raise optuna.exceptions.TrialPruned()

    # סגירת הריצה ב-W&B והחזרת התוצאה ל-Optuna
    run.finish()
    return val_acc

In [74]:
# יצירת המחקר
study = optuna.create_study(direction="maximize")

# הרצה לבדיקה ראשונית - 5 דקות בלבד (300 שניות)
study.optimize(objective, timeout=300) 

print("--- הבדיקה הסתיימה בהצלחה! ---")
print("הפרמטרים הטובים ביותר שמצאנו ב-5 דקות:", study.best_params)

[I 2026-01-02 13:02:35,527] A new study created in memory with name: no-name-b63628b2-ab88-4585-83ca-7418a14a4700
wandb: WARNING Using a boolean value for 'reinit' is deprecated. Use 'return_previous' or 'finish_previous' instead.


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


batch_loss,▅▄▁▂▂▄▂▆▄▃▂▁▅▃▄▃▂▂▃▃▄▃▂▃▂▂▁█▄▆▂▂▁▁▂▃▁▁▂▃
epoch,▁▃▅▆█
train_loss,█▄▃▂▁
val_acc,▁█▇██
val_batch_loss,▅▇▆▇▅▅▇█▂▅▃▃▃▅▂▂▁▁▄▂▁█▁▂▁▃▁▄▄▁▁▂▁▅▂▃▃▃▂▂
val_loss,█▂▂▁▁
batch_loss,0.0237
epoch,4
train_loss,0.14442
val_acc,0.94002
val_batch_loss,0.03188


[I 2026-01-02 13:12:01,662] Trial 0 finished with value: 0.9400206825232679 and parameters: {'lr': 0.0012011055202177864, 'optimizer': 'SGD', 'batch_size': 16}. Best is trial 0 with value: 0.9400206825232679.


--- הבדיקה הסתיימה בהצלחה! ---
הפרמטרים הטובים ביותר שמצאנו ב-5 דקות: {'lr': 0.0012011055202177864, 'optimizer': 'SGD', 'batch_size': 16}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

############# FOR LATER USE ###################

def imshow(tensor):
    """
    Converts a normalized PyTorch tensor back to a visual image and displays it.
    
    The function reverses the ImageNet normalization by multiplying by the 
    standard deviation and adding the mean. It also clips values to the 
    [0, 1] range to ensure compatibility with matplotlib.
    
    Args:
        tensor (torch.Tensor): A normalized image tensor of shape (C, H, W).
    """
    img = tensor.numpy().transpose((1, 2, 0))
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    
    img = std * img + mean
    img = np.clip(img, 0, 1)
    
    plt.imshow(img)
    plt.show()